In [1]:
import json

import evaluate
import torch as tt
import pandas as pd

from typing import Any, Dict, Union

from nltk.tokenize import sent_tokenize
from tqdm import tqdm_notebook
from datasets import Dataset, load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, PreTrainedModel, PreTrainedTokenizer
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

In [2]:
# models:
tokenizer = T5Tokenizer.from_pretrained("ai-forever/ruT5-base")
model = T5ForConditionalGeneration.from_pretrained("ai-forever/ruT5-base").to(tt.device("cuda:0"))

# metrics:
bleu4 = evaluate.load("bleu")
sbleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/lib/python3/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
[nltk_data] Downloading package wordn

In [3]:
option_id_dict = {
    'A': 0, 'B': 1, 'C': 2, 'D': 3
}

def to_new_format(example: dict[str, Union[str, list[str]]]) -> str:
  inp, label = '', ''
  example["options_ru"] = [option for option in example["options_ru"] if option]
  right_answer = example['options_ru'][option_id_dict[example['answer']]]

  right_answer = right_answer.replace('"', "'")

  qtext_orig = example["question"].lower()
  if ("not true" in qtext_orig) or ("false" in qtext_orig) or ("n't true" in qtext_orig) or ("untrue" in qtext_orig):
      if ("not false" in qtext_orig) or ("n't false" in qtext_orig):
          inp += example['article_ru'] + " " + "ВОПРОС: Какое высказывание СООТВЕТСТВУЕТ тексту? "
      else:
          inp += example['article_ru'] + " " + "ВОПРОС: Какое высказывание НЕ СООТВЕТСТВУЕТ тексту? "
  else:
      inp += example['article_ru'] + " " + "ВОПРОС: Какое высказывание СООТВЕТСТВУЕТ тексту? "

  inp += f'ПРАВИЛЬНЫЙ ОТВЕТ: "{right_answer}".'
  inp += 'НЕПРАВИЛЬНЫЕ ВАРИАНТЫ ОТВЕТА: '

  options = example["options_ru"]
  options = [
      option.replace('"', "'") for option in options if option != right_answer
  ]
  options = [
      f'"{option}"' for option in options
  ]
  label = "; ".join(options)

  distractors_len = len(tokenizer(label)["input_ids"])
    
  return {"inp": inp, "right_answer": right_answer, "distractors": label, "distractors_len": distractors_len}

In [4]:
with open("tf_dataset_pretty_filtered.json", 'r', encoding="utf8") as inp:
    tf_dataset = json.load(inp)

tf_dataset_train, tf_dataset_val, tf_dataset_test = tf_dataset["train"], tf_dataset["val"], tf_dataset["test"]
tf_dataset_train = Dataset.from_list(tf_dataset_train)
tf_dataset_val = Dataset.from_list(tf_dataset_val)
tf_dataset_test = Dataset.from_list(tf_dataset_test)

tf_dataset_train = tf_dataset_train.map(to_new_format)
tf_dataset_val = tf_dataset_val.map(to_new_format)
tf_dataset_test = tf_dataset_test.map(to_new_format)

Map:   0%|          | 0/3288 [00:00<?, ? examples/s]

Map:   0%|          | 0/175 [00:00<?, ? examples/s]

Map:   0%|          | 0/187 [00:00<?, ? examples/s]

In [5]:
pd.Series(tf_dataset_train["distractors_len"]).describe()

count    3288.000000
mean       47.546533
std        11.146655
min        21.000000
25%        40.000000
50%        46.000000
75%        54.000000
max       136.000000
dtype: float64

In [6]:
MAX_LEN = pd.Series(tf_dataset_train["distractors_len"]).quantile(0.99)
MAX_LEN

81.0

In [7]:
def model_predict(text: str, model: PreTrainedModel) -> str:
    input_ = tokenizer([text], return_tensors="pt")
    output = model.generate(
        input_["input_ids"].to(tt.device("cuda:0"))
    )
    return tokenizer.batch_decode(output)[0]

In [8]:
print(tf_dataset_test[42]["inp"])

Почему люди скрещивают пальцы, сталкиваясь с трудностями нерелигиозных людей? Почему мы виним чёрного кота, когда видим низкий балл теста?
Эти привычки называются волшебным мышлением и можно найти везде в повседневной жизни. Например, человек хочет чего - то плохого для раздражающего коллеги. Или родители молятся о безопасном бою, когда их сын уходит. 
Психологическая помощь
Изучая эти привычки, ученые решили найти причину, по которой люди верят в магические силы. Психологи имеют свой собственный подход к этому вопросу. 
"Я думаю, что отчасти это потому, что мы постоянно подвергаемся воздействию наших собственных мыслей, и поэтому, скорее всего, переоценим их связь с внешними событиями. "Сказала Эмили Пронин, психолог в префиксе = st1 /US.
Для людей, которые не уверены в своих способностях или медленных действиях, волшебное мышление может быть большой помощью, объяснил доктор Дэниел Вегнер, профессор психологии Гарварда. Чувство, что их собственные мысли могут контролировать вещи, може

In [9]:
print(tf_dataset_test[42]["distractors"])

"Джастин Барретт, эволюционист, отрицает, что магическое мышление является продуктом адаптации."; "У людей со слабой верой больше шансов выжить в соперничестве."; "Адаптационисты думают, что мы рождаемся с тенденцией верить в наше волшебное мышление."


In [10]:
predict_model = model_predict(tf_dataset_test[42]["inp"], model)
print(predict_model)

/home/user/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> <extra_id_0>,,,,,,,,,,,,,,,,,,


In [11]:
print(tf_dataset_test[100]["inp"])

У тебя есть какие-нибудь замечательные планы на предстоящие зимние каникулы? Вот замечательные фильмы, чтобы убить время. ВОПРОС: Какое высказывание СООТВЕТСТВУЕТ тексту? ПРАВИЛЬНЫЙ ОТВЕТ: "Сиротка - это комедия об Эстер, усыновленной доброй семьёй.".НЕПРАВИЛЬНЫЕ ВАРИАНТЫ ОТВЕТА: 


In [12]:
print(tf_dataset_test[100]["distractors"])

"500 дней лета - роман с счастливым концом."; "Мы идем показывать красивые пейзажи во время путешествия молодых пар."; "Древнее предсказание майя произойдет в 2012 году."


In [13]:
predict_model = model_predict(tf_dataset_test[100]["inp"], model)
print(predict_model)

<pad> <extra_id_0> :  :::::::::::::::


In [14]:
predict_model = model_predict(tf_dataset_test[100]["inp"], model)
print(predict_model)

<pad> <extra_id_0> :  :::::::::::::::


In [15]:
help(model.generate)

Help on method generate in module transformers.generation.utils:

generate(inputs: Optional[torch.Tensor] = None, generation_config: Optional[transformers.generation.configuration_utils.GenerationConfig] = None, logits_processor: Optional[transformers.generation.logits_process.LogitsProcessorList] = None, stopping_criteria: Optional[transformers.generation.stopping_criteria.StoppingCriteriaList] = None, prefix_allowed_tokens_fn: Optional[Callable[[int, torch.Tensor], List[int]]] = None, synced_gpus: Optional[bool] = None, assistant_model: Optional[ForwardRef('PreTrainedModel')] = None, streamer: Optional[ForwardRef('BaseStreamer')] = None, negative_prompt_ids: Optional[torch.Tensor] = None, negative_prompt_attention_mask: Optional[torch.Tensor] = None, **kwargs) -> Union[transformers.generation.utils.GenerateDecoderOnlyOutput, transformers.generation.utils.GenerateEncoderDecoderOutput, transformers.generation.utils.GenerateBeamDecoderOnlyOutput, transformers.generation.utils.GenerateBe

In [16]:
def get_metric_inputs_seq2seq(
    input_batch: list[str], #label_batch: list[str],
    model: PreTrainedModel, tokenizer: PreTrainedTokenizer
) -> list[str]:
    input_batch_ = tokenizer(input_batch, return_tensors="pt", padding=True)["input_ids"].to(tt.device("cuda:0"))
    # label_batch_ = tokenizer(label_batch, return_tensors="pt", padding=True)["input_ids"]

    # output_length = label_batch_.shape[-1]

    with tt.no_grad():
        output_batch = model.generate(input_batch_, max_length=MAX_LEN)

    output = [
        sent.replace("<pad>", " ").replace("</s>", " ").strip() for sent in tokenizer.batch_decode(output_batch) # 
    ]
    
    del input_batch_
    del output_batch
    # del label_batch_
    tt.cuda.empty_cache()

    return output

def compute_metrics(output: list[str], label_batch: list[str]) -> dict:
    metric_dict = {
        "bleu": bleu4.compute(predictions=output, references=[[label] for label in label_batch]),
        "sbleu": sbleu.compute(predictions=output, references=[[label] for label in label_batch]),
        "rouge": rouge.compute(predictions=output, references=label_batch),
        "meteor": meteor.compute(predictions=output, references=label_batch)
    }
    return metric_dict

In [17]:
model_inputs = tf_dataset_test["inp"][:16]
right_answers = tf_dataset_test["right_answer"][:16]
labels = [item.replace('\n', '').replace('  ',' ').replace('  ',' ').strip() for item in tf_dataset_test["distractors"][:16]]

In [18]:
predict_model = get_metric_inputs_seq2seq(model_inputs, model, tokenizer)

In [19]:
labels

['"Самые великие земледелцы пустыни - это люди."; "Пустыни быстро растут."; "Размеры пустыни постоянно меняются."',
 '"Нельсон Мандела не был его оригинальным именем."; "Нельсон Мандела был назван своим учителем."; "Нельсон Мандела основал свою собственную юридическую фирму до того, как получил степень юриста."',
 '"Детям нехорошо веселиться летом."; "Детям будет скучно читать программы"; "Учителям не нужно помогать детям анализировать уроки."',
 '"Фестиваль Гластонбери работает на прибыльной основе."; "Джеймс Браун и Джосс Стоун родились в бедных семьях."; "В 1970 году на фестивале Гластонбери можно бесплатно пообедать на ферме."',
 '"Университет находится в центре города."; "Студенты могут жить на улице."; "Университет не заинтересован в удовлетворении растущих потребностей общества."',
 '"Все жертвы получили легкие ранения в результате аварии."; "Спасатели были доставлены в больницу для посещения жертв."; "Раненые вскоре оправятся от полученных ранений."',
 '"Тигр Вудс был смешанным

In [20]:
predict_model

['<extra_id_0> :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::',
 '<extra_id_0>,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,',
 '<extra_id_0>,, <extra_id_2>',
 '<extra_id_0>, в том числе, в мире.,,,,!!!!!! <extra_id_12> <extra_id_13>',
 '<extra_id_0>,, в, в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в',
 '<extra_id_0>, за, за, за, за, за, за, за.... <extra_id_1> <extra_id_2>',
 '<extra_id_0>,, в, в и, в, в и,, в и,, в и,, в',
 '<extra_id_0>, что мы, и, <extra_id_2>',
 '<extra_id_0> :, как и в',
 '<extra_id_0>, в и в',
 '<extra_id_0>, в,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,',
 '<extra_id_0>., в, в,,,,,,,,,!!!!!!!!!!!!!',
 '<extra_id_0>, что мы, <extra_id_2>',
 '<extra_id_0>. <extra_id_1>. <extra_id_2>',
 '<extra_id_0>,',
 '<extra_id_0> :) <extra_id_1> :) <extra_id_2>']

In [21]:
right_answers

['В пустыне нет живых существ.',
 'Нельсон Мандела изучал этот закон без перерыва в течение 50 лет.',
 'Летние программы могут помочь детям.',
 'Билеты на Фестиваль Гластонбери 2004 года были очень востребованы, несмотря на высокую цену.',
 'Культурная жизнь университета очень богата.',
 'К счастью, никто не получил слишком серьезных травм во время аварии.',
 'Появление Тайгера Вудса на сцене гольфа изменило отношение к этому спорту в США.',
 'Ложись спать сразу после горячей ванны.',
 'Мозги нужно упражняться.',
 'Рестораны в Испании думают о метафоре, чтобы удовлетворить потребности людей.',
 'В некоторых случаях правдивая информация помогает пациентам справляться со своей болезнью.',
 'Это фермер застрелил крокодила.',
 'Мы не такие высокие в конце дня, как в начале.',
 'С помощью компьютера, используемого в нашей повседневной жизни, мы можем сделать кое-что попроще, чем раньше.',
 'Вы можете наслаждаться акробатическими шоу каждый день в 19 часов на открытом воздухе.',
 'Более двух

In [22]:
compute_metrics(predict_model, labels)

{'bleu': {'bleu': 0.0,
  'precisions': [0.050243111831442464, 0.0, 0.0, 0.0],
  'brevity_penalty': 1.0,
  'length_ratio': 1.0730434782608695,
  'translation_length': 617,
  'reference_length': 575},
 'sbleu': {'score': 0.1407467864723911,
  'counts': [31, 0, 0, 0],
  'totals': [617, 601, 585, 569],
  'precisions': [5.024311183144246,
   0.08319467554076539,
   0.042735042735042736,
   0.021968365553602813],
  'bp': 1.0,
  'sys_len': 617,
  'ref_len': 575},
 'rouge': {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0},
 'meteor': {'meteor': 0.023340126465681454}}

In [23]:
BATCH_SIZE = 1
N_STEPS = (len(tf_dataset_val) // BATCH_SIZE) + 1

metrics_val = []

for i in tqdm_notebook(range(N_STEPS), total=N_STEPS):
    slice = tf_dataset_val[i*BATCH_SIZE:(i+1)*BATCH_SIZE]

    if slice["inp"]:
        output = get_metric_inputs_seq2seq(slice["inp"], model, tokenizer)

        distractors = [item.replace('\n', '').replace('  ',' ').replace('  ',' ').strip() for item in slice["distractors"]]

        if "ВОПРОС: Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?" in slice["inp"][0]:
            question = "ВОПРОС: Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?"
        else:
            question = "ВОПРОС: Какое высказывание СООТВЕТСТВУЕТ тексту? "

        try:
            metric = compute_metrics(output, distractors)
            metrics_val.append({
                "article": slice["article_ru"][0],
                "right_answer": slice["right_answer"][0],
                "question": question,
                "distractors": distractors[0],
                "output": output[0],
    
                "bleu": metric["bleu"]["bleu"],
                "sbleu": metric["sbleu"]["score"],
                "rouge1": metric["rouge"]["rouge1"],
                "rouge2": metric["rouge"]["rouge2"],
                "rougeL": metric["rouge"]["rougeL"],
                "rougeLsum": metric["rouge"]["rougeLsum"],
                "meteor": metric["meteor"]["meteor"],
    
                "article_orig": slice["article"][0],
                "question_orig": slice["question"][0],
                "options_orig": slice["options"][0],
                "right_answer_orig": slice["answer"][0]
            })
        except ZeroDivisionError:
            metrics_val.append({
                "article": slice["article_ru"][0],
                "right_answer": slice["right_answer"][0],
                "question": question,
                "distractors": distractors[0],
                "output": output[0],
    
                "bleu": 0,
                "sbleu": 0,
                "rouge1": 0,
                "rouge2": 0,
                "rougeL": 0,
                "rougeLsum": 0,
                "meteor": 0,
    
                "article_orig": slice["article"][0],
                "question_orig": slice["question"][0],
                "options_orig": slice["options"][0],
                "right_answer_orig": slice["answer"][0]
            })

/tmp/ipykernel_103386/3942987125.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(N_STEPS), total=N_STEPS):


  0%|          | 0/176 [00:00<?, ?it/s]

In [25]:
metrics_val = pd.DataFrame(metrics_val)

In [26]:
metrics_val.describe()

,bleu,sbleu,rouge1,rouge2,rougeL,rougeLsum,meteor
count,175.0,175.000000,175.0,175.0,175.0,175.0,175.000000
mean,0.0,0.484962,0.0,0.0,0.0,0.0,0.019417
std,0.0,0.479414,0.0,0.0,0.0,0.0,0.018901
min,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
25%,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
50%,0.0,0.418443,0.0,0.0,0.0,0.0,0.015060
75%,0.0,0.736774,0.0,0.0,0.0,0.0,0.030770
max,0.0,2.053632,0.0,0.0,0.0,0.0,0.088933


In [27]:
metrics_val.to_excel("T5Metrics-TF-Baseline-val.xlsx", engine="openpyxl")